### Land ice analysis requires ATL03 and ATL06 data

In [ ]:
import os
os.chdir('..')
from notebooks import config
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import pyproj
import icepyx as ipx
import geopandas as gpd
import pointCollection as pc
from utils.imgShow import imgShow
from utils.geotif_io import readTiff
from utils.read_HDF5_ATL03 import read_HDF5_ATL03
from utils.get_ATL03_x_atc import get_ATL03_x_atc
from utils.mutliple_is2_requests import multiple_is2_requests
from utils.atl06beam_to_dict import atl06beam_to_dict


In [ ]:
path_img = config.root + '/data/rs_img/kunlun_S3A_20191225T045036_20191225T045336.tif'


In [ ]:
## read remote sensing image
s3_img, s3_img_info = readTiff(path_img)
# s3_extent = rio.plot.plotting_extent(s3_img_src)
# s3_img_src.crs
## read alt03 data
rgt = "0081"
cycle = "06"
beam = 'gt2l'
# read the atl03 data with Tyler's ATL03 reader:
atl03_file=glob.glob(os.path.join(config.root, f'data/icesat2/atl03_06_demo/processed_ATL03_*{rgt}{cycle}*.h5'))[0]
atl03_mds, atl03_attrs, atl03_beams =read_HDF5_ATL03(atl03_file, VERBOSE=True)
atl03_beam = atl03_mds[beam]
get_ATL03_x_atc(atl03_mds, atl03_attrs, atl03_beams)
atl03_x, atl03_y = np.array(pyproj.Proj(32644)(atl03_beam['heights']['lon_ph'], atl03_beam['heights']['lat_ph']))

### read atl06 data
atl06_file=glob.glob(os.path.join(config.root, f'data/icesat2/atl03_06_demo/processed_ATL06*{rgt}{cycle}*.h5'))[0]
atl06_beam=atl06beam_to_dict(atl06_file, beam, index=None, epsg=32644)

In [ ]:
plt.figure()
imgShow(s3_img, extent=s3_img_info['extent'], color_bands=(0, 1, 2), clip_percent=5)
plt.plot(atl03_x, atl03_y, color='r')
plt.title('Mosaic of tibet for Kun Lun mountain')


In [ ]:
fig=plt.figure(figsize=(10,8))
## fig.1: atl03 photon heights
ax1 = fig.add_subplot(221)
ax1.plot(atl03_beam['heights']['x_atc'], atl03_beam['heights']['h_ph'],'k.',markersize=0.25, label='all photons') 
h_leg=ax1.legend()
ax1.set_xlabel('x_atc, m')
ax1.set_ylabel('h, m')
# ax.set_xlim([3950000, 4000000])
# ax.set_ylim([4000, 6500])

### fig.2: atl03 good-quality heights
LMH=atl03_beam['heights']['signal_conf_ph'][:,3] >= 2 # good quality
ax1.plot(atl03_beam['heights']['x_atc'][LMH], atl03_beam['heights']['h_ph'][LMH],'g.',markersize=0.5, label='flagged photons')
ax1.legend()

### fig.3: atl03 backgroud(noisy heights)
ax2 = fig.add_subplot(222, sharex=ax1, sharey=ax1)
BG=(atl03_beam['heights']['signal_conf_ph'][:,3] ==0) | (atl03_beam['heights']['signal_conf_ph'][:,3] ==1)          ## backgroud(noisy...)
ax2.plot(atl03_beam['heights']['x_atc'][BG], atl03_beam['heights']['h_ph'][BG],'k.',markersize=0.25, label='Background')
ax2.legend()
### atl03 TEP points
ax3 = fig.add_subplot(223, sharex=ax1, sharey=ax1)
TEP=(atl03_beam['heights']['signal_conf_ph'][:,3] <-1)    ## 
ax3.plot(atl03_beam['heights']['x_atc'][TEP], atl03_beam['heights']['h_ph'][TEP],'b.',markersize=0.25, label='TEP')
ax3.legend()
### atl06 points
ax4 = fig.add_subplot(224, sharex=ax1, sharey=ax1)
ax4.plot(atl06_beam['x_atc'], atl06_beam['h_li'],'r.', markersize=0.25,label='ATL06')
ax4.legend()

plt.show()


In [ ]:

fig=plt.figure(figsize=[10, 8])
### fig.1: plot atl03 photon heights
ax1=fig.add_subplot(221)
plt.plot(atl03_beam['heights']['x_atc'], atl03_beam['heights']['h_ph'],'k.', markersize=0.5, label='atl03 photon heights')
ax1.legend()
plt.title('atl03 photon heights')

### fig.2: plot the atl06 segment heights 
ax2 = fig.add_subplot(222, sharex=ax1, sharey=ax1)
hl=plt.plot(atl06_beam['x_atc'], atl06_beam['h_li'], color='b', markersize=0.5, label='atl06 segment height')

### fig.3: plot the atl06 segment quality -> ->good/bad
good=atl06_beam['atl06_quality_summary']==0
ax3 = fig.add_subplot(223, sharex=ax1, sharey=ax1)
plt.plot(atl06_beam['x_atc'][~good], atl06_beam['h_li'][~good],'y.', label='atl06 bad segment (flag=1)', markersize=0.5)
plt.plot(atl06_beam['x_atc'][good], atl06_beam['h_li'][good],'r.', label='atl06 good segment (flag=0)', markersize=0.5)
ax3.legend()
plt.title('atl06 segment heights')

### fig.4: plot atl06 segments difference, less -> better.
ax4 = fig.add_subplot(224, sharex=ax1, sharey=ax1)
delta_h_seg = min_seg_difference(atl06_beam)
cm=plt.scatter(atl06_beam['x_atc'], atl06_beam['h_li'], 4, c=delta_h_seg, vmin=0, vmax=1, cmap='autumn', zorder=3);

plt.colorbar(cm, ax=[ax1, ax2, ax3, ax4], label='min_seg_difference, m')
plt.title('atl06 min_seg_difference');


### Comparison between 2 cycels


In [ ]:
D_2l={}
D_2r={}
rgt = "0325"
atl06_files = glob.glob(config.root + f'/data/icesat2/atl03_06(demo)/processed_ATL06_*_{rgt}*_004*.h5')
map_fig=plt.figure(figsize=(8,4))
ax=map_fig.add_subplot(111)
# iterate over the repeat cycles
for cycle in ['02','06']:
    for filename in atl06_files:
        try:
            # read the left/right-beam data
            D_2l[filename]=atl06beam_to_dict(filename,'/gt2l', index=None, epsg=32644)
            D_2r[filename]=atl06beam_to_dict(filename,'/gt2r', index=None, epsg=32644)
            # plot the locations in the previous plot
        except Exception as e:
            print(f'filename={filename}, exception={e}')

for filename, Di in D_2l.items():
    # Plot atl06 segments that have quality_summary==0 (good quality)
    hl=plt.plot(Di['x_atc'][Di['atl06_quality_summary']==0], Di['h_li'][Di['atl06_quality_summary']==0], '.', markersize=2, label=f"cycle={Di['cycle']}")

plt.legend()
plt.xlabel('x_atc')
plt.ylabel('elevation');


In [ ]:
# specify the rgt here:
# rgt = "0325"
for cycle in ['02', '06']:
    for filename in atl06_files:
        try:
            # read the left/right-beam data
            D_2l[filename]=atl06beam_to_dict(filename,'/gt2l', index=None, epsg=32644)
            D_2r[filename]=atl06beam_to_dict(filename,'/gt2r', index=None, epsg=32644)
        except Exception as e:
            print(f'filename={filename}, exception={e}')

plt.figure(figsize=[8, 10])
## rs image
ax1=plt.subplot(311)
imgShow(s3_img, extent=s3_img_info['extent'], color_bands=(0, 1, 2), clip_percent=5)
ax2=plt.subplot(312)
ax3=plt.subplot(313, sharex=ax2)
for filename, Dl in D_2l.items():
    Dr=D_2r[filename]
    good_dl = Dl['atl06_quality_summary'] == 0
    good_dr = Dr['atl06_quality_summary'] == 0
    ## Dl plot
    hl = ax2.plot(Dl['x_atc'][good_dl], Dl['h_li'][good_dl],'.', \
                            label=f"cycle={Dl['cycle']}", markersize=2)
    ax1.plot(Dl['x'][::10], Dl['y'][::10], '.', color=hl[0]._color, markersize=2)
    ## Dr plot
    ax2.plot(Dr['x_atc'][good_dr], Dr['h_li'][good_dr], '+', \
                    label=f"cycle={Dr['cycle']}", color=hl[0]._color, markersize=2)
    ax1.plot(Dr['x'][::10], Dr['y'][::10], '+', color=hl[0]._color, markersize=2)
    ax3.plot(Dr['x_atc'][::10], Dr['y_atc'][::10],'.', \
                                        color=hl[0]._color, markersize=2)
    ## Dl plot
    ax3.plot(Dl['x_atc'][::10], Dl['y_atc'][::10],'+', \
                                        color=hl[0]._color,markersize=2)

for ax in [ax2, ax3]:
    ax.legend()
    ax.set_xlabel('x_atc')
ax2.set_ylabel('elevation')
ax3.set_ylabel('y_atc')


### Correction of the repeat elevations for cross-track slope
### !!! lager errors after correction???


In [ ]:
fig=plt.figure(figsize=(8,8))
ax1=fig.add_subplot(211)
ax2=fig.add_subplot(212, sharex=ax1, sharey=ax1)
color_dict={}
for filename, Dl in D_2l.items():
    Dr=D_2r[filename]
    good_dl=Dl['atl06_quality_summary']==0
    good_dr=Dr['atl06_quality_summary']==0
    hl=ax1.plot(Dl['x_atc'][good_dl], Dl['h_li'][good_dl],'.', markersize=3, label=f"cycle={Dl['cycle']}")
    # save the color for this cycle so we can use it again
    color_dict[int(Dl['cycle'])]=hl[0]._color
    # for Dl correction, !!may not suitable for moutainous region
    atc_corr_dl = Dl['y_atc']*Dl['dh_fit_dy']
    Dl_h_correct = Dl['h_li']-atc_corr_dl   # corrected Dl heights
    ax2.plot(Dl['x_atc'][good_dl], Dl_h_correct[good_dl], '.', markersize=3, color=color_dict[int(Dl['cycle'])])
    ax1.plot(Dr['x_atc'][good_dr], Dr['h_li'][good_dr], '+', markersize=4, color=color_dict[int(Dl['cycle'])])
   # for Dl correction, !!may not suitable for moutainous region
    atc_corr_dr = Dr['y_atc']*Dr['dh_fit_dy']   # for Dr correction
    Dr_h_correct = Dr['h_li']-atc_corr_dr   # corrected Dr heights
    ax2.plot(Dr['x_atc'][good_dr], Dr_h_correct[good_dr], '+', markersize=4, color=color_dict[int(Dl['cycle'])])
 
ax1.legend()
ax1.set_title('Before correction')
ax2.set_title('After correction')
